In [ ]:
from pathlib import Path
from tqdm import tqdm
from utils import read_images, save_min_projection_imgs, plot_plate, predict_masks, extract_stats, copy_csv_results, save_summary_stats
import torch

# Check CUDA availability
print("CUDA Available:", torch.cuda.is_available())

In [ ]:
# Define your data directory (folder containing the subfolders storing your plate images)
PARENT_FOLDER = Path("./data")

# Define the output resolution of your plate plots and username
RESOLUTION = 300
USERNAME = "Andrew"

# Define your output directory (folder containing the results of the analysis)
RESULTS_FOLDER = Path("./output")

# Merge results_folder and username into a Path
results_directory = RESULTS_FOLDER / USERNAME

In [ ]:
# Initialize an empty list to store subfolder names
subfolder_list = []

# Iterate over subdirectories in the parent folder
for subfolder in PARENT_FOLDER.iterdir():
    if subfolder.is_dir() and "4X" not in str(subfolder):
        subfolder_list.append(subfolder.name)

for folder in tqdm(subfolder_list):
    # Build the path containing the input images
    directory_path = PARENT_FOLDER.joinpath(folder)
    
    # The following function will read all the images contained within the directory_path above
    # and store them grouped by well_id.
    images_per_well = read_images(directory_path)
    
    # Specify the output directory path to store all resulting output data for this particular plate
    directory = Path(f"./output/{USERNAME}")
    output_directory = directory.joinpath(folder)
    
    # Specify the output directory for minimum intensity projections (MIN_projections)
    min_output_directory = output_directory.joinpath("MIN_projections")
    
    # Perform and save minimum intensity projections
    save_min_projection_imgs(images_per_well, output_dir=min_output_directory)
    
    # Save min_proj plate view
    plot_plate(resolution=RESOLUTION, output_path=f"{output_directory}/{folder}_MIN_proj.tif", img_folder_path=min_output_directory, show_fig=False)
    
    # Specify the output directory for YOLOv8 predictions (masks and bounding boxes)
    predictions_output_directory = output_directory.joinpath("predictions")
    
    # Predict masks and bounding boxes and store them under predictions_output_directory
    predict_masks(input_folder=min_output_directory, output_dir=predictions_output_directory)
    
    # Save predictions plate view
    plot_plate(resolution=RESOLUTION, output_path=f"{output_directory}/{folder}_predictions.tif", img_folder_path=predictions_output_directory, show_fig=False)
    
    # Extract statistics from YOLOv8 predictions
    all_props_df = extract_stats(input_folder=min_output_directory)
    
    # Save resulting stats in a .csv file
    all_props_df.to_csv(f"{output_directory}/{folder}.csv", index=False)
    
# Copies .csv files containing per organoid results into a per_organoid_stats folder within results_directory
copy_csv_results(results_directory)

# Copies .csv files containing per organoid results into a per_organoid_stats folder within results_directory
save_summary_stats(results_directory)
